In [1]:
import os
import seaborn as sb
import matplotlib.pyplot as plt

print(os.listdir("."))

['.ipynb_checkpoints', 'EDA.ipynb']


In [2]:
from pathlib import Path
path = Path("../")

In [3]:
%%time
import pandas as pd
data = path / "GAMMA_hackathon/customer_data.csv"
data = pd.read_csv(data)


CPU times: user 6.6 s, sys: 562 ms, total: 7.17 s
Wall time: 7.16 s


In [4]:
data.describe()

,customer_id,months_since_joined,referred_friends,transaction_count,transaction_value,clv,total_reward_points,earned_reward_points
count,1.143614e+07,1.143614e+07,1.143614e+07,1.143614e+07,1.143614e+07,1.143614e+07,1.143614e+07,1.143614e+07
mean,6.086582e+05,2.210391e+01,9.669775e-01,2.936650e+01,1.504392e+02,3.600556e+03,5.648918e+03,2.941982e+02
std,2.847427e+05,1.978018e+01,9.877185e-01,1.471019e+01,1.645431e+02,5.712062e+03,1.374896e+04,5.555236e+02
min,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.067390e+05,6.000000e+00,0.000000e+00,1.900000e+01,5.816020e+01,4.291993e+02,2.759791e+02,4.600000e+01
50%,6.195790e+05,1.600000e+01,1.000000e+00,2.600000e+01,8.558780e+01,1.130037e+03,1.068997e+03,1.000000e+02
75%,8.241590e+05,3.400000e+01,2.000000e+00,3.600000e+01,1.718362e+02,4.080206e+03,4.421000e+03,2.660000e+02
max,1.204356e+06,9.300000e+01,9.000000e+00,1.410000e+02,1.929116e+03,6.423930e+04,4.863967e+05,1.238400e+04


In [5]:
for col in data.columns:
    print(col, " sample ", data[col].sample().values)

customer_id  sample  [578738]
month  sample  ['2018-01-01']
months_since_joined  sample  [41]
referred_friends  sample  [3]
transaction_count  sample  [32.]
transaction_value  sample  [203.91900691]
clv  sample  [221.37031076]
total_reward_points  sample  [43.99800949]
earned_reward_points  sample  [102.]
cluster  sample  ['B']
churned  sample  [ True]


In [6]:
data.shape
data["customer_id"].unique().size

836833

In [7]:
data = data.sort_values(by="month")

In [11]:
can_choose = data.groupby("customer_id").tail(1)
can_choose = can_choose.loc[can_choose["churned"] == False]

In [12]:
import tqdm

tqdm.tqdm.pandas()


In [ ]:
%%time
train = pd.DataFrame(data)

def shift_churn(df):
    df["churned"] = df["churned"].shift(-1)
    return df
    
train["churned"] = train.groupby("customer_id").progress_apply(shift_churn)["churned"]
train = data.groupby("customer_id").tail(2).groupby("customer_id").head(1)


 89%|████████▉ | 744885/836833 [06:51<00:49, 1864.88it/s]

In [ ]:
print(last.shape)
print(train.shape)

In [ ]:
def vectorize_data(x):
    train = pd.DataFrame()
    train["msj"] = (x["months_since_joined"] - x["months_since_joined"].mean()) / x["months_since_joined"].std()
    train["rf"] = (x["referred_friends"] - x["referred_friends"].mean()) / x["referred_friends"].std()
    train["tc"] = (x["transaction_count"] - x["transaction_count"].mean()) / x["transaction_count"].std()
    train["tv"] = (x["transaction_value"] - x["transaction_value"].mean()) / x["transaction_value"].std()
    train["trp"] = (x["total_reward_points"] - x["total_reward_points"].mean()) / x["total_reward_points"].std()
    train["erp"] = (x["earned_reward_points"] - x["earned_reward_points"].mean()) / x["earned_reward_points"].std()
    
    clusters = pd.get_dummies(x["cluster"])
    
    return pd.concat([train, clusters], axis=1), x["churned"].apply(int)


train, label = vectorize_data(train)

In [ ]:
predict, _ = vectorize_data(can_choose)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

clf = MLPClassifier(verbose=True).fit(train.values, label)
#clf = RandomForestClassifier(verbose=True, n_estimators=200).fit(train.values, label)

In [ ]:
import lightgbm

parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'logloss',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 138,
    'learning_rate': 0.05,
    'verbose': 0
}

t = lightgbm.Dataset(train, label)

model = lightgbm.train(parameters,
                       t,
                       num_boost_round=1000)

In [ ]:
lightgbm.plot_importance(model)

In [ ]:
preds = model.predict(predict.values)
train_preds = model.predict(train.values)

In [ ]:
preds = clf.predict_proba(predict.values)[:,1]
train_preds = clf.predict_proba(train.values)[:, 1]

print(preds)

In [ ]:
from sklearn.metrics import log_loss 

print(log_loss(label, train_preds))

In [ ]:
user_values = pd.DataFrame()
user_values["customer_id"] = can_choose["customer_id"]
user_values["value"] = preds * can_choose["clv"].values
user_values["prob"] = preds
user_values["clv"] = can_choose["clv"].values
user_values["churn"] = can_choose["churned"].values
user_values["C"] = can_choose["cluster"]

In [ ]:
users = 10000
best_x_value = user_values.sort_values(by="value").tail(users)
best_x_prob = user_values.sort_values(by="clv").tail(users)
best_x_clv = user_values.sort_values(by="prob").tail(users)

In [ ]:
plt.title("Groups")
plot = "value"
for name, g in user_values.groupby("C"):
    sb.distplot(g[plot], label=name)

plt.legend()

In [ ]:
plt.figure(figsize=(20, 10))
sb.distplot(user_values["clv"])

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(3, 1, 1)
sb.distplot(best_x_value["value"])
plt.subplot(3, 1, 2)

sb.distplot(best_x_prob["prob"])
plt.subplot(3, 1, 3)

sb.distplot(best_x_clv["clv"])

In [ ]:
sb.distplot(preds)

In [ ]:
submission = pd.DataFrame()
submission["customer_id"] = best_x_value["customer_id"]

submission.to_csv("E.csv")